In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd #a library to make the data more structured55r56y6y

In [2]:
df = pd.read_csv('data/text-preprocessing.csv')
df = df.drop(columns=['Unnamed: 0'])

In [3]:
df_new = df[df['tweet_tokens_stemmed_string'].notnull()]

In [4]:
df_new['tweet_tokens_stemmed_string'].head()

0    lomba poster ilmiah energi baru deadline janua...
1                 elaahhh ngomong energi baru tv biruu
2    daerah pencil membutuhkam listrik manfaat ener...
3    tarik materi energi baru baek ganti nama propa...
4    aneh inget jatropa alias minyak jarak bbrp jat...
Name: tweet_tokens_stemmed_string, dtype: object

In [5]:
text = df_new['tweet_tokens_stemmed_string']
text_list =  [i.split() for i in text]
#text_list =  [i for i in text]
print(len(text_list))

109404


In [6]:
#masi salah hasil frekuensi nya
# NLTK calc frequency distribution
from nltk.probability import FreqDist
def freqDist_wrapper(text):
    return FreqDist(text)

df_new['tweet_tokens_stemmed_string'] = df_new['tweet_tokens_stemmed_string'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
df_new['tweet_tokens_stemmed_string'].head().apply(lambda x : x.most_common())

Frequency Tokens : 



0    [( , 11), (a, 10), (e, 6), (i, 6), (s, 5), (r,...
1    [( , 5), (e, 3), (a, 3), (h, 3), (n, 3), (g, 3...
2    [(a, 7), ( , 6), (e, 5), (r, 4), (i, 4), (m, 4...
3    [(a, 10), ( , 8), (r, 5), (i, 5), (e, 4), (t, ...
4    [(a, 12), ( , 12), (r, 8), (e, 7), (b, 6), (i,...
Name: tweet_tokens_stemmed_string, dtype: object

In [7]:
import gensim
#Create Bigram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(text_list, min_count=10)
trigram = Phrases(bigram[text_list])
for idx in range(len(text_list)):
    for token in bigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)
    for token in trigram[text_list[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_list[idx].append(token)

In [8]:
from gensim import corpora, models
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(text_list)
dictionary.filter_extremes(no_below=5, no_above=0.2) 
#no_below (int, optional) – Keep tokens which are contained in at least no_below documents.
#no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).
print(dictionary)

Dictionary<16354 unique tokens: ['deadline', 'derajat', 'ilmiah', 'januari', 'lomba']...>


In [9]:
#https://radimrehurek.com/gensim/tut1.html 
#build corpus
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
#The function doc2bow converts document (a list of words) into the bag-of-words format
'''The function doc2bow() simply counts the number of occurrences of each distinct word, 
converts the word to its integer word id and returns the result as a sparse vector. 
The sparse vector [(0, 1), (1, 1)] therefore reads: in the document “Human computer interaction”, 
the words computer (id 0) and human (id 1) appear once; 
the other ten dictionary words appear (implicitly) zero times.'''
print(len(doc_term_matrix))
print(doc_term_matrix[100])
tfidf = models.TfidfModel(doc_term_matrix) #build TF-IDF model
corpus_tfidf = tfidf[doc_term_matrix]

109404
[(35, 1), (188, 1)]


In [10]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
#function to compute coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100, random_state=1)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [11]:
start=1
limit=21
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step)
#show graphs
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 3))

Num Topics = 1  has Coherence Value of 0.517
Num Topics = 2  has Coherence Value of 0.539
Num Topics = 3  has Coherence Value of 0.436
Num Topics = 4  has Coherence Value of 0.435
Num Topics = 5  has Coherence Value of 0.423
Num Topics = 6  has Coherence Value of 0.402
Num Topics = 7  has Coherence Value of 0.424
Num Topics = 8  has Coherence Value of 0.386
Num Topics = 9  has Coherence Value of 0.41
Num Topics = 10  has Coherence Value of 0.369
Num Topics = 11  has Coherence Value of 0.332
Num Topics = 12  has Coherence Value of 0.363
Num Topics = 13  has Coherence Value of 0.346
Num Topics = 14  has Coherence Value of 0.352
Num Topics = 15  has Coherence Value of 0.321
Num Topics = 16  has Coherence Value of 0.322
Num Topics = 17  has Coherence Value of 0.321
Num Topics = 18  has Coherence Value of 0.327
Num Topics = 19  has Coherence Value of 0.328
Num Topics = 20  has Coherence Value of 0.313


In [ ]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=7, random_state=1) #num topic menyesuaikan hasil dari coherence value paling tinggi

In [ ]:
for idx, topic in model.print_topics():
    print('Topic: {} Word: {}'.format(idx+1, topic))
    # yg belum remove Yang, yg,trus lower text

Topic: 1 Word: 0.030*"bangkit_listrik" + 0.021*"capai_target" + 0.020*"bangkit" + 0.018*"target" + 0.016*"baur" + 0.016*"panas_bumi" + 0.016*"capai" + 0.013*"pln" + 0.013*"listrik" + 0.012*"kejar_target"
Topic: 2 Word: 0.052*"target_baur" + 0.043*"menteri_bumn" + 0.016*"dana" + 0.016*"negara_maju" + 0.015*"tenaga_biogas" + 0.014*"alas" + 0.014*"untung" + 0.013*"anggar" + 0.012*"harga_jual" + 0.011*"subsidi"
Topic: 3 Word: 0.013*"wujud" + 0.009*"rencana" + 0.009*"renewable_energy" + 0.009*"capai_persen" + 0.009*"milik_potensi" + 0.008*"cepat_transisi" + 0.008*"beli_listrik" + 0.008*"pln" + 0.008*"transisi" + 0.008*"indonesia_timur"
Topic: 4 Word: 0.022*"erick_thohir" + 0.012*"kembang" + 0.011*"erick" + 0.010*"thohir" + 0.009*"investor" + 0.009*"panel_surya" + 0.008*"yantie_pln" + 0.008*"realisasi" + 0.008*"udien_pln" + 0.007*"tekan"
Topic: 5 Word: 0.033*"menteri_esdm" + 0.011*"menteri" + 0.011*"esdm" + 0.010*"pltu" + 0.009*"tinggi_indonesia" + 0.009*"bahan_bakar" + 0.008*"buka_peluang" 

In [ ]:
import pandas as pd
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 5)]) #ubah2 yg ini
# df=pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words_topic_20.csv")
df = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P']).to_excel("data/lda_model_topic_7.xlsx")
print(df)


None


In [ ]:
#ini dilakukan jika module belum tersedia
!pip install pyLDAvis
#!apt-get -qq install -y pyLDAvis


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#import gensim
import pyLDAvis
import pyLDAvis.gensim_models;pyLDAvis.enable_notebook()
data = pyLDAvis.gensim_models.prepare(model, corpus_tfidf, dictionary)
print(data)
pyLDAvis.save_html(data, 'data/lda-gensim_clean_kualitatif7.html')

c:\Users\asus\OneDrive - Universitas Pertamina\TA\tugas-akhir\venv\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.035384 -0.238855       1        1  17.356396
0      0.204753  0.095364       2        1  16.756378
5      0.169429 -0.143969       3        1  16.706684
4      0.066384  0.074683       4        1  14.383824
6     -0.086128  0.246700       5        1  13.199615
3      0.001788  0.005644       6        1  11.577854
1     -0.320843 -0.039567       7        1  10.019249, topic_info=                  Term         Freq        Total Category  logprob  loglift
7140            pln_id  2086.000000  2086.000000  Default  30.0000  30.0000
1895       target_baur  1409.000000  1409.000000  Default  29.0000  29.0000
3787      menteri_bumn  1165.000000  1165.000000  Default  28.0000  28.0000
357       menteri_esdm  1310.000000  1310.000000  Default  27.0000  27.0000
783    bangkit_listrik  1375.000000  1375.000000  Default  26.0000  26.0000
...            